# Emotion Analysis with Deep Learning using BERT

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

## Exploratory Data Analysis and Preprocessing

Use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv("smile-annotations-final.csv", 
                names = ["id", "text", "category"])
df.set_index("id", inplace=True)

In [3]:
df.head()

,text,category
id,,
"pel @mitpress @monoskop http://t.co/fxp5kjuGV2""",nocode,NaN
612168095252267008,@stiveshouse @MillenniumArt @Tate_StIves @Trem...,happy
611634051502485504,I miss Cornwall so much. #ff @FernPitCafe @Pen...,not-relevant
610763295042224128,£10 off @accessart http://t.co/0biFQaXTQg memb...,not-relevant
611529306917969922,@britishmuseum @Ophiolatrist Spain? Egypt?,nocode


In [4]:
df.text.iloc[1]

'@stiveshouse @MillenniumArt @Tate_StIves @Tremenheere ooh... Please let us know how it is!!!'

In [6]:
#check class balance
df.category.value_counts()

nocode               1554
happy                1123
not-relevant          211
angry                  56
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [7]:
#remove tweets with nocode and multiple categories
df = df[df.category.str.contains("\|") == False]

In [8]:
df

,text,category
id,,
612168095252267008,@stiveshouse @MillenniumArt @Tate_StIves @Trem...,happy
611634051502485504,I miss Cornwall so much. #ff @FernPitCafe @Pen...,not-relevant
610763295042224128,£10 off @accessart http://t.co/0biFQaXTQg memb...,not-relevant
611529306917969922,@britishmuseum @Ophiolatrist Spain? Egypt?,nocode
615107089619394560,Enjoying the amazing collections @britishmuseu...,happy
...,...,...
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy
613294681225621504,@britishmuseum Upper arm guard?,nocode
615246897670922240,@MrStuchbery @britishmuseum Mesmerising.,happy


In [9]:
df = df[df.category != "nocode"]

In [10]:
df

,text,category
id,,
612168095252267008,@stiveshouse @MillenniumArt @Tate_StIves @Trem...,happy
611634051502485504,I miss Cornwall so much. #ff @FernPitCafe @Pen...,not-relevant
610763295042224128,£10 off @accessart http://t.co/0biFQaXTQg memb...,not-relevant
615107089619394560,Enjoying the amazing collections @britishmuseu...,happy
611239436739870720,@NationalGallery Beautiful painting. Thank you.,happy
...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy


In [11]:
df.category.value_counts()

happy           1123
not-relevant     211
angry             56
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [12]:
possible_labels = df.category.unique()

In [13]:
label_dict = {}
for i, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = i

In [14]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [15]:
df["label"] = df.category.replace(label_dict)
df

<ipython-input-15-4be5050e9544>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = df.category.replace(label_dict)


,text,category,label
id,,,
612168095252267008,@stiveshouse @MillenniumArt @Tate_StIves @Trem...,happy,0
611634051502485504,I miss Cornwall so much. #ff @FernPitCafe @Pen...,not-relevant,1
610763295042224128,£10 off @accessart http://t.co/0biFQaXTQg memb...,not-relevant,1
615107089619394560,Enjoying the amazing collections @britishmuseu...,happy,0
611239436739870720,@NationalGallery Beautiful painting. Thank you.,happy,0
...,...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0


## Training/Validation Split

In [16]:
from sklearn.model_selection import train_test_split

### Note that the dataset is imbalanced. Hence, use the stratified approach to split data.


In [17]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                 df.label.values,
                                                 test_size = 0.15,
                                                 random_state = 17,
                                                 stratify = df.label.values)

In [18]:
df["data_type"] = ["not_set"]*df.shape[0]

<ipython-input-18-7de485427970>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["data_type"] = ["not_set"]*df.shape[0]


In [19]:
df

,text,category,label,data_type
id,,,,
612168095252267008,@stiveshouse @MillenniumArt @Tate_StIves @Trem...,happy,0,not_set
611634051502485504,I miss Cornwall so much. #ff @FernPitCafe @Pen...,not-relevant,1,not_set
610763295042224128,£10 off @accessart http://t.co/0biFQaXTQg memb...,not-relevant,1,not_set
615107089619394560,Enjoying the amazing collections @britishmuseu...,happy,0,not_set
611239436739870720,@NationalGallery Beautiful painting. Thank you.,happy,0,not_set
...,...,...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1,not_set
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0,not_set
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0,not_set


In [20]:
df.loc[X_train, "data_type"] = "train"
df.loc[X_val, "data_type"] = "val"

/Users/pratyushaghosh/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [21]:
df.groupby(["category", "label", "data_type"]).count()

text
category     label data_type      
angry        2     train        48
                   val           8
disgust      3     train         5
                   val           1
happy        0     train       954
                   val         169
not-relevant 1     train       179
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

## Loading Tokenizer and Encoding the Data

In [22]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [23]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",
                                         do_lower_case = True,
                                         )

In [24]:
encoded_data_train = tokenizer.batch_encode_plus(df[df.data_type =="train"].text.values,
                                                add_special_tokens = True,
                                                return_attention_mask=True,
                                                pad_to_max_length = True,
                                                max_length=256,
                                                return_tensors="pt")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/pratyushaghosh/opt/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [25]:
encoded_data_val = tokenizer.batch_encode_plus(df[df.data_type =="val"].text.values,
                                                add_special_tokens = True,
                                                return_attention_mask=True,
                                                pad_to_max_length = True,
                                                max_length=256,
                                                return_tensors="pt")

In [26]:
input_ids_train = encoded_data_train["input_ids"]
attention_mask_train = encoded_data_train["attention_mask"]
labels_train = torch.tensor(df[df.data_type == "train"].label.values)

In [27]:
input_ids_val = encoded_data_val["input_ids"]
attention_mask_val = encoded_data_val["attention_mask"]
labels_val = torch.tensor(df[df.data_type == "val"].label.values)

In [28]:
dataset_train = TensorDataset(input_ids_train, attention_mask_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_mask_val, labels_val)

In [29]:
len(dataset_train)

1243

In [30]:
len(dataset_val)

220

## Setting up BERT Pretrained Model

In [31]:
from transformers import BertForSequenceClassification

In [50]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                     output_hidden_states = False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Creating Data Loaders

In [52]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [66]:
batch_size = 32 #4 or 32
dataloader_train = DataLoader(dataset_train,
                             sampler = RandomSampler(dataset_train),
                             batch_size = batch_size
                             )

In [67]:
dataloader_val = DataLoader(dataset_val,
                             sampler = RandomSampler(dataset_val),
                             batch_size = 32
                             )

## Setting Up Optimizer and Scheduler

In [68]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [69]:
optimizer = AdamW(model.parameters(),
                 lr = 1e-5,
                 eps = 1e-8,
                 )

In [70]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps=0,
                                           num_training_steps=len(dataloader_train)*epochs
                                           )

## Defining Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [71]:
import numpy as np

In [72]:
from sklearn.metrics import f1_score

In [73]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average="weighted")

In [74]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        print(f"Class: {label_dict_inverse[label]}")
        print(f"Accuracy: {len(y_preds[y_preds == label])}/{len(y_true)}\n")

## Creating the Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [75]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [76]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(device)

cpu


In [77]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [78]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc="Epoch {:1d}".format(epoch), leave=False, disable = False)
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {"input_ids": batch[0],
                 "attention_mask": batch[1],
                 "labels": batch[2]}
        
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({"training_loss": "{:.3f}".format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(), f"BERT_ft_epoch{epoch}.model")
    
    tqdm.write("\nEpoch {epoch}")
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f"Training Loss: {loss_train_avg}")
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f"Validation Loss: {val_loss}")
    tqdm.write(f"F1 score (weighted): {val_f1}")


Epoch {epoch}
Training Loss: 1.3803274753766182



Validation Loss: 1.0407326136316573
F1 score (weighted): 0.6674690348212199



Epoch {epoch}
Training Loss: 0.9113041208340571



Validation Loss: 0.8102074606078011
F1 score (weighted): 0.6674690348212199



Epoch {epoch}
Training Loss: 0.7495779456236423


...


Validation Loss: 0.4663142604487283
F1 score (weighted): 0.8213957759412305



Epoch {epoch}
Training Loss: 0.3225650791174326



Validation Loss: 0.4732463146959032
F1 score (weighted): 0.8220558148499995



Epoch {epoch}
Training Loss: 0.3071837988801492



Validation Loss: 0.4816071689128876
F1 score (weighted): 0.8164873546665775



## Loading and Evaluating the Model

In [79]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [80]:
model.to(device)
pass

In [86]:
#model.load_state_dict(
#    torch.load('finetuned_bert_epoch_1_gpu_trained.model',
#              map_location=torch.device('cpu')))

In [82]:
_, predictions, true_vals = evaluate(dataloader_val)

In [83]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 168/169

Class: not-relevant
Accuracy: 31/32

Class: angry
Accuracy: 8/8

Class: disgust
Accuracy: 1/1

Class: sad
Accuracy: 5/5

Class: surprise
Accuracy: 4/5

